In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import time
import smtplib, ssl
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate



In [8]:
spac_info = load_spac_info("/Users/ZhenxinLei/MyWork/quantamental/spac/active_spacs_clean.csv")
tickers = list(spac_info['Symbol'])
price = load_hist_data(tickers)
print(price.tail())

                                          Company  Symbol    IPO Date  \
0                 8i Enterprises Acquisition Corp   JFKKU   3/28/2019   
1               Acamar Partners Acquisition Corp.    ACAM   2/22/2019   
2               Ace Convergence Acquisition Corp.  ACEV.U   7/28/2020   
3                           AGBA Acquisition Ltd.    AGBA   5/14/2019   
4                Alberton Acquisition Corporation    ALAC  10/24/2018   
5                 Alussa Energy Acquisition Corp.    ALUS  11/26/2019   
6                          AMCI Acquisition Corp.    AMCI  11/16/2018   
7    Amplitude Healthcare Acquisition Corporation    AMHC  11/20/2019   
8                    Andina Acquisition Corp. III    ANDA   1/29/2019   
9         Apex Technology Acquisition Corporation    APXT   9/17/2019   
10                        Artius Acquisition Inc.    AACQ   7/14/2020   
11              ARYA Sciences Acquisition Corp II    ARYB    6/5/2020   
12             ARYA Sciences Acquisition Corp III  

[*********************100%***********************]  168 of 168 completed

34 Failed downloads:
- PTK.U: No data found, symbol may be delisted
- ACEV.U: No data found, symbol may be delisted
- CHAQ.U: No data found, symbol may be delisted
- LEAP.U: No data found, symbol may be delisted
- SBG.U: No data found, symbol may be delisted
- FAII.U: No data found, symbol may be delisted
- STPK.U: No data found, symbol may be delisted
- BFT.U: No data found, symbol may be delisted
- CLA.U: No data found, symbol may be delisted
- HZAC.U: No data found, symbol may be delisted
- BSN.U: No data found, symbol may be delisted
- GOAC.U: No data found, symbol may be delisted
- NPAU: No data found, symbol may be delisted
- TWND.U: No data found, symbol may be delisted
- GSAH.U: No data found, symbol may be delisted
- EAGL6: No data found, symbol may be delisted
- AONE.U: No data found, symbol may be delisted
- NSH.U: No data found, symbol may be delisted
- DGNR.U: No data found, symbol may be delisted
- 

In [46]:
def load_spac_info(path_to_csv="active_spacs_clean.csv"):
    df = pd.read_csv(path_to_csv)
    print(df)
    return df

def load_hist_data(tickers, start=None, end=None, interval='1d',valid_data_window= 10):
    price_df =None
    if start is None or end is None:
        price_df = yf.download(tickers, period='1Y', interval = interval)
    else:
        price_df = yf.download(tickers, start,end, interval = interval)
        
    invalid_tickers =price_df['Close'].iloc[-valid_data_window-1].isnull()
    invalid_tickers = invalid_tickers[invalid_tickers==True]

    price_df = price_df.drop(invalid_tickers.index.tolist(), axis=1, level=1)

    return price_df

'''
return dataframe columns name like "[('col_1', 'ticker_1'),('col_1','ticker_2'),...]"
'''
def gen_factor_1_df(price_df,close_col='Close',volume_col = 'Volume', look_back=10, ma_window = 10):
    df = pd.DataFrame(index = price_df.index)
    
    rolling_mean=price_df.xs(close_col, axis=1,level=0,drop_level=False).rolling(window = ma_window).mean()
    #return_cols = rolling_mean/rolling_mean.shift(look_back)-1
    close_price = price.xs(close_col, axis=1,level=0,drop_level=False)
    return_cols = close_price/rolling_mean.shift(look_back)-1
    return_cols =return_cols.rename(columns={close_col:'price_chg_pct'})
    df = df.join(return_cols)
    
    #yesterday return 
    close_price = price.xs(close_col, axis=1,level=0,drop_level=False)
    return_cols = close_price/close_price.shift(1)-1
    return_cols =return_cols.rename(columns={close_col:'ytday_price_chg_pct'})
    
    df = df.join(return_cols)
    
    #print(df.columns)
    
    vol_rolling_mean=price_df.xs(volume_col, axis=1,level=0,drop_level=False).rolling(window = ma_window).mean()
    volume = price.xs(volume_col, axis=1,level=0,drop_level=False)
    vol_chg_pct = volume/vol_rolling_mean.shift(look_back)-1
    vol_chg_pct =vol_chg_pct.rename(columns={volume_col:'vol_chg_pct'})
    df = df.join(vol_chg_pct)
    
    avg_price =price_df[['Close','Open','High','Low']].mean(axis=1,level=1)
    volume = price_df['Volume']
    volume_in_usd =avg_price*volume
    #print(volume_in_usd.columns,avg_price.columns )
    for col_name in volume_in_usd.columns:
        volume_in_usd = volume_in_usd.rename(columns={col_name:('volume_in_usd',col_name)})
    df = df.join(volume_in_usd)

    return df

def get_fct1_filter1_tickers(tickers, fct1_df):
    abnormal_tickers=[]
    for ticker in tickers:
        is_abnormal_return =fct1_df[('price_chg_pct',ticker)].iloc[-1]> 0.05
        is_abnormal_volume = fct1_df[('vol_chg_pct',ticker)].iloc[-1]>0.30
        if is_abnormal_return and is_abnormal_volume:
            #print( ticker, is_abnormal_return)
            abnormal_tickers.append(ticker)
    
    return abnormal_tickers


def get_fct1_filter2_tickers(tickers, fct1_df):
    abnormal_tickers=[]
    for ticker in tickers:
        is_abnormal_return = (fct1_df[('price_chg_pct',ticker)].iloc[-1]<=0.03) & (fct1_df[('price_chg_pct',ticker)].iloc[-1]>=-0.03)
        is_abnormal_volume = fct1_df[('vol_chg_pct',ticker)].iloc[-1]>0.50
        if is_abnormal_return and is_abnormal_volume:
            #print( ticker, is_abnormal_return)
            abnormal_tickers.append(ticker)
    
    return abnormal_tickers

In [47]:
price.tail()

Adj Close                                                          \
                ACAM   AGBA   ALAC  ALUS   AMCI  AMHC    ANDA    APXT   ARYA   
Date                                                                           
2020-09-11     10.25  10.30  10.66  9.92  10.38  9.99  10.225  10.250  10.90   
2020-09-14     10.20  10.30  10.78  9.90  10.38  9.97  10.250  10.225  10.90   
2020-09-15     10.25  10.30  10.78  9.93  10.36  9.93  10.212  10.180  10.90   
2020-09-16     10.36  10.35  10.78  9.93  10.55  9.89  10.210  10.200  10.81   
2020-09-17     10.37  10.35  10.73  9.93  10.34  9.90  10.210  10.740  10.75   

                    ...    Volume                                          \
              ARYB  ...      THBR  THCA   THCB   TOTA       TREB     TRNE   
Date                ...                                                     
2020-09-11  10.350  ...   66900.0     0   7200   2400  1165000.0  2890900   
2020-09-14  10.450  ...  105000.0   400  16200   3600    47000.0  3069200   
2020-09-15  10.900  ...  100800.0   600   4700   3100    72600.0  1477100   
2020-09-16  11.240  ...   77400.0   500   5400  29600    22800.0   877500   
2020-09-17  11.054  ...    4700.0  4000  65700    900    19400.0  1455600   

                                                
              TZAC     VMAC       WPF    ZGYHU  
Date                                            
2020-09-11       0  30500.0  109300.0      0.0  
2020-09-14     400   5000.0  160000.0   2775.0  
2020-09-15  353000   5500.0  237800.0  11000.0  
2020-09-16   64400  24100.0  432300.0   4102.0  
2020-09-17    3700   8000.0  135800.0   8598.0  

[5 rows x 720 columns]

In [57]:
df = gen_factor_1_df(price)
df.tail()
abnormal_tickers = get_fct1_filter2_tickers(price['Close'].columns, df)

to_send ={}
for ticker in abnormal_tickers:
    
    tmp = pd.DataFrame()
    to_send[ticker]=tmp
    #tmp = df[[('price_chg_pct',ticker), ('vol_chg_pct',ticker),('volume_in_usd',ticker)]]
    tmp['price ma change(%)']= (df[('price_chg_pct',ticker)]*100).apply(lambda x: '{:,.2f}%'.format(x))
    tmp['ytday price change(%)']= (df[('ytday_price_chg_pct',ticker)]*100).apply(lambda x: '{:,.2f}%'.format(x))
    tmp['volume change(%)'] =(df[('vol_chg_pct',ticker)]*100).apply(lambda x: '{:,.2f}%'.format(x))
    tmp['volume USD($MM)'] = (df[('volume_in_usd',ticker)]/1000000).apply(lambda x: '${:,.2f}MM'.format(x))
    tmp['Close']=price['Close'][ticker].apply(lambda x: '{:,.2f}'.format(x))
    
    #has_target = (spac_info[spac_info['Symbol']==ticker]['Merger Target?']).values
    #ipo_date = spac_info[spac_info['Symbol']==ticker]['IPO Date'].values
    liq_date = spac_info[spac_info['Symbol']==ticker][['Symbol','Merger Target?','IPO Date', 'Liquidation Date']]
    #print(liq_date)
    #print(tmp.tail(5))
    #print('\n')
print(abnormal_tickers)   
#print(to_send['ACAM'].tail(20))

/Users/ZhenxinLei/riverrocktech/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


['ALAC', 'ALUS', 'BRLI', 'CCAC', 'CIIC', 'DFPH', 'ESSC', 'GRSVU', 'JIH', 'KBLM', 'LOAC', 'LSAC', 'MFAC', 'MLAC', 'MNCL', 'ROCH', 'SFTW', 'ZGYHU']


In [215]:


def send_email(to_send):
    filename ='sample.txt'
    file_full_name = './'+filename
    with open(file_full_name,'w') as outfile:
        for ticker, df  in to_send.items():
            outfile.write('\n======== %s ========\n'%(ticker))
            header = spac_info[spac_info['Symbol']==ticker][['Symbol','Merger Target?','IPO Date', 'Liquidation Date']]
            header.to_string(outfile)
            outfile.write('\n')
            to_send[ticker].tail(10).to_string(outfile)
            outfile.write('\n\n')
            
    try:
        server = smtplib.SMTP("smtp.gmail.com","587")
        #self.server.ehlo() # Can be omitted
        server.starttls()
        #self.server.ehlo() # Can be omitted
        server.login("heytechnologies@gmail.com", 'Zhxlei87!')

        message = MIMEMultipart()
        message['From'] = "heytechnologies@gmail.com"
        message['Bcc'] = COMMASPACE.join(['zhenxinlei@gmail.com'])
        message['Subject']='test sample'

        sent=0
        retry=0
        while sent==0:
            try:
                
                #filename = "document.pdf"  # In same directory as script

                # Open PDF file in binary mode
                with open(file_full_name, "rb") as attachment:
                    # Add file as application/octet-stream
                    # Email client can usually download this automatically as attachment
                    part = MIMEApplication(
                        attachment.read(),
                        Name=filename
                    )

                # Encode file in ASCII characters to send by email    
                #encoders.encode_base64(part)

                # Add header as key/value pair to attachment part
                part['Content-Disposition'] = 'attachment; filename="%s"' % (filename)
        

                # Add attachment to message and convert message to string
                message.attach(part)
                text = message.as_string()

                server.sendmail('heytechnologies@gmail.com',['zhenxinlei@gmail.com'],text)
                sent+=1
            except Exception as e:
                time.sleep(1) #TODO not good
                retry +=1
                if retry == 2:
                    raise Exception ("Failed to send Email after retry %s"%e)
                    break
                #self.__logger.log(logging.WARN,"Failed to send email: %s"%e)
    except Exception as e:
        print(e)
    finally:
        server.quit()
    
send_email(to_send)
    

In [24]:
rolling_mean=price.xs(['Close'], axis=1,level=0,drop_level=False).rolling(window = 10).mean()
close_price = price.xs(['Close'], axis=1,level=0,drop_level=False)
return_cols = close_price/rolling_mean.shift(10)-1
print(rolling_mean['Close']['SBE'].tail(5))
print(close_price['Close']['SBE'].tail(5))
print(return_cols['Close']['SBE'].tail(5))

Date
2020-09-11    10.2945
2020-09-14    10.3115
2020-09-15    10.3365
2020-09-16    10.6465
2020-09-17    11.0465
Name: SBE, dtype: float64
Date
2020-09-11    10.35
2020-09-14    10.37
2020-09-15    10.45
2020-09-16    13.30
2020-09-17    14.25
Name: SBE, dtype: float64
Date
2020-09-11    0.009411
2020-09-14    0.012448
2020-09-15    0.021755
2020-09-16    0.301688
2020-09-17    0.393848
Name: SBE, dtype: float64


In [53]:
a =[]
b=[1,2]
c=[3,4]
a.append(b)
a.append(c)

In [55]:
print(a)

[[1, 2], [3, 4]]
